## EECS 453/551
# Video background subtraction using SVD

In this exercise we will discover how the SVD can magically estimate the background in a video even when there is a lot of noise and missing data.
___

## Before we begin

Run the following code cell by clicking it and either

* choosing Cell -> Run in the toolbar
* Pressing Ctrl+Enter

This will install and load the Julia packages needed for today's demo.

In [ ]:
# install required packages if not already present:
Pkg.add("MAT")      # load .mat files
Pkg.add("PyPlot")   # plotting tool
Pkg.add("Interact") # interact with plots

# load required packages:
using MAT, Interact, PyPlot

# set graphics options:
PyPlot.svg(true) # not your grandma's raster graphics
set_cmap("gray") # we want grayscale video, not heatmap

filename = "lobby.mat"
# filename = "xylophone.mat"

You may wish to take a few minutes to familiarize yourself with the notebook interface. Choose Help -> User Interface Tour for a brief introduction.

In Julia, you can type ? followed by any object (variable, function, module) to get its description. Try typing `?MAT` into a new code cell and running it.

## Objective

The objective of this exercise is to take as an input the video you have just seen and return as an output the background (i.e. without the people moving for the lobby video).

### Task 1

Find in the function below the portion where the variable `backgroundVec` is computed. Note how the SVD is used to compute it. Examine how the `residualVec` variable is computed and note that

    footageVec = backgroundVec + residualVec

Hence if we successfully are able to estimate the background, `residualVec` should contain:

1. People moving around
2. Nothing (empty)

**Do you expect 1. or 2. to be true? Why?**

In [ ]:
"""
    process(footage) -> background,residual
Use SVD to separate input footage into background (static objects)
and residual (fleeting objects).
"""
function process(footage)
    
    (m,n,numFrames) = size(footage)
    
    # Each movie frame is a 128x160 array.
    # Reshape each frame into a single column vector
    # to ease manipulation:
    footageVec = reshape(footage,m*n,numFrames)
    
    ###################################
    # Process using SVD
    r = 1 # rank of "background" is 1

    # U,S,V will contain first r components of SVD:
    (U,S,V) = svds(footageVec,nsv=r)
    
    # take first r components as background
    backgroundVec = (U*diagm(S))*V'
    
    # calculate residual
    residualVec = footageVec - backgroundVec    
    ###################################

    # reshape background and residual into footage dimensions
    background = reshape(backgroundVec,size(footage))
    residual = reshape(residualVec,size(footage))
    
    return background,residual
end

### Task 2

Run the following code cell. You should see a slider above a plot with two subplots. On the left is the original footage; on the right is the background as estimated by SVD.

**Is it working?**

In [ ]:
### load 128x160x650 array containing video data:
footage = matread(filename)["MovMat"]
m,n,numFrames = size(footage)

# perform SVD-based background subtraction:
background,residual = process(footage)

# display results:
fig = figure(figsize=[10;6])
@manipulate for frameNumber in 1:numFrames; withfig(fig) do     
        
        # footage:
        subplot(1,2,1)
        imshow(footage[:,:,frameNumber])
        axis("off")
        title("Footage")
        
        # background:
        subplot(1,2,2)
        imshow(background[:,:,frameNumber])
        axis("off")
        title("Background (SVD processing)")
    end
end

### Task 3: adding noise

The movie you just processed using SVD was relatively noise-free. Let us start adding noise and removing entries and see how well SVD does.

Study the function `addnoise()` in the following cell. **What is the code doing mathematically?** Now **run the cell** to add the function to your workspace.

Enter a value of 0.1 for $ \sigma $ in the next cell down, then run it.

**Do you see the movie get noisier? Is the SVD able to extract the background?**

* Now repeat for $ \sigma = 0.5 $, $ \sigma = 1 $ and $ \sigma = 2 $. **How does the SVD do?**

In [ ]:
"""
Add noise to footage.

σ is noise standard deviation.

(type symbol with "\sigma<TAB>")
"""
function addnoise(footage,σ)
    # Interpret this mathematically!
    footage = (footage + σ*rand(size(footage)))
    return footage
end   

In [ ]:
### load 128x160x650 array containing video data:
footage = matread(filename)["MovMat"]
m,n,numFrames = size(footage)

############################
σ =  # Enter your value here

# Add noise:
footage = addnoise(footage,σ)
############################

# perform background subtraction on noisy footage:
background,residual = process(footage)

# display results:
fig = figure(figsize=[10;6])
@manipulate for frameNumber in 1:numFrames; withfig(fig) do 
        
        # footage:
        subplot(1,2,1)
        imshow(footage[:,:,frameNumber])
        axis("off")
        title("Footage")
        
        # background:
        subplot(1,2,2)
        imshow(background[:,:,frameNumber])
        axis("off")
        title("Background (SVD processing)")
    end
end

### Task 4: removing entries

Now we will corrupt the data by randomly removing a portion of the entries.

Study `corrupt()` in the cell below. **What is it doing to the input footage?** Now **run the cell** to add `corrupt()` to your workspace.

Enter a value of 0.9 for `p` in the next cell down, then run it.

**Does the movie have missing entries? Is the SVD successfully separating the background?**

Now repeat for `p = 0.75`, `p = 0.5`, `p = 0.25`, `p = 0.1`. **Do you see the movie have more blank entries? Is the SVD still working?**

In [ ]:
"""
Corrupt footage by removing entries.

Each entry is removed with probability `p`.
"""
function corrupt(footage,p)
    # Interpret this mathematically!
    # Type ?rand if you aren't sure what rand() does.
    mask = rand(size(footage)) .< p

    # Mask some entries
    footage = footage.*mask
    return footage
end

In [ ]:
### load 128x160x650 array containing video data:
footage = matread(filename)["MovMat"]
m,n,numFrames = size(footage)

############################
σ = 0.1
p =  # Enter your value here

# Add noise:
footage = addnoise(footage,σ)

# Delete entries:
footage = corrupt(footage,p)
############################

# perform background subtraction on noisy footage:
background,residual = process(footage)

# display results:
fig = figure(figsize=[10;6])
@manipulate for frameNumber in 1:numFrames; withfig(fig) do 
        
        # footage:
        subplot(1,2,1)
        imshow(footage[:,:,frameNumber])
        axis("off")
        title("Footage")
        
        # background:
        subplot(1,2,2)
        imshow(background[:,:,frameNumber])
        axis("off")
        title("Background (SVD processing)")
    end
end

### Task 5

Return to the very first code cell, uncomment the line

```julia
filename = "xylophone.mat",
```

and run the cell again.

Now repeat Tasks 3 and 4. **Does the SVD still work?**